### Scrape links from TED

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os 
import json 

In [2]:
## send the request
#url = "https://www.ted.com/talks?language=zh-cn"
## chinese taiwan
url = "https://www.ted.com/talks?language=zh-tw"
result = requests.get(url)
print(result.status_code)

200


In [3]:
## get page number
c = result.content
soup = BeautifulSoup(c,"lxml")
pages =  soup.find('div',{'class':'results__pagination'}).find('div')
page_num = pages.findAll('a')[-2].getText()
page_num = int(page_num)
print(page_num)

61


In [4]:
## create doc_list to hold all links 
doc_list = {'title':[],'author':[],'date':[],'link':[]}

## loop through each page
for p in range(1,page_num+1):
    page_url = url + '&page=' + str(p) + '&sort=newest'
    result = requests.get(page_url)
    #print(result.status_code)
    c = result.content
    soup = BeautifulSoup(c,"lxml")
    docs = soup.find('div',{'id':'browse-results'}).findAll('div',{'class':'col'})

    for doc in docs:
        author = doc.find('h4',{'class':'h12 talk-link__speaker'}).getText()
        link = 'https://www.ted.com' + doc.find('a').get('href')
        title = doc.find('h4',{'class':'h9 m5'}).getText().strip(' \t\n\r').replace('\n','').replace('\r',' ')
        date = doc.find('div',{'class':'media__message'}).find('span',{'meta__item'}).find('span',{'meta__val'}).getText().strip(' \t\n\r').replace('\n','')

        doc_list['title'].append(title)
        doc_list['author'].append(author)
        doc_list['date'].append(date)
        doc_list['link'].append(link)
        ## end of  inner loop
## end of outer loop 
print(len(doc_list['link']))

2183


In [7]:
## export links file 
df = pd.DataFrame(doc_list)
df['status']= 0
df.to_csv('Ted_links.csv')
df.ix[0:5]

,author,date,link,title,status
0,Manwar Ali,Oct 2016,https://www.ted.com/talks/manwar_ali_inside_th...,窺探前激進聖戰士的內心,0
1,Pico Iyer,Oct 2016,https://www.ted.com/talks/pico_iyer_the_beauty...,未知的美麗,0
2,Ellen Jorgensen,Oct 2016,https://www.ted.com/talks/ellen_jorgensen_what...,關於 CRISPR，你該知道什麼？,0
3,Sam Harris,Sep 2016,https://www.ted.com/talks/sam_harris_can_we_bu...,我們能打造不會失控的人工智慧？,0
4,Oded Shoseyov,Sep 2016,https://www.ted.com/talks/oded_shoseyov_how_we...,我們如何駕馭大自然隱藏的超能力,0
5,J.D. Vance,Sep 2016,https://www.ted.com/talks/j_d_vance_america_s_...,被遺忘的美國勞工階層家庭與他們的困境,0
